# Execution tutorial

This tutorial covers the basics of executing a quantum circuit using Classiq and the two different ways of executing a quantum program: Via [Classiq platform](platform.classiq.io) or directly through the Python SDK.

For this, we will start by synthesizing the following example, from the [synthesis tutorial](https://docs.classiq.io/latest/explore/tutorials/basic_tutorials/the_classiq_tutorial/synthesis_tutorial/):

In [1]:
from classiq import *


@qfunc
def main(x: Output[QNum[3]], y: Output[QNum]) -> None:
    allocate(x)
    hadamard_transform(x)
    y |= x**2 + 1

qprog = synthesize(main)

This quantum program evaluates the function $y(x) = x^2 + 1$, for all integers $x \in [0,7]$. To execute a quantum program and save its results in the Python SDK, use `execute`:

In [2]:
results = execute(qprog)

The outputs of the quantum circuit can be obtained through the attributes `counts` and `parsed_counts` from `results.result_value()`.

In [3]:
print("Output of the executing using counts: ", results.result_value().counts)

print("Output of the executing using parsed_counts: ", results.result_value().parsed_counts)

Output of the executing using counts:  {'000101010': 280, '110010111': 224, '010001100': 234, '001010011': 284, '000010001': 248, '100101110': 253, '000001000': 254, '011010101': 271}
Output of the executing using parsed_counts:  [{'x': 3, 'y': 10}: 284, {'x': 2, 'y': 5}: 280, {'x': 5, 'y': 26}: 271, {'x': 0, 'y': 1}: 254, {'x': 6, 'y': 37}: 253, {'x': 1, 'y': 2}: 248, {'x': 4, 'y': 17}: 234, {'x': 7, 'y': 50}: 224]


As we can see, the results from the execution are the same but differ in format: 

* `counts` will output a `dict` containing the bitstrings measured followed by its number of counts.
* `parsed_counts` will output a `list` of `dicts`, each containing the values of `x` and `y` followed by its number of counts.

These two ways of reading the results can be used in different.

## Backend selection

The backend of an execution is the hardware choice where the quantum program is executed. It can be classical hardware, i.e., a simulator, or real hardware. To select a specific backend, it is necessary to import its correct Backend Preferences from `classiq.execution`. Check the different [Cloud Providers](https://docs.classiq.io/latest/user-guide/execution/cloud-providers/) and their Backend Preferences for execution.

For example, let's execute the latter program on the state vector simulator from Classiq. Since Classiq provides this backend, we will use `ClassiqBackendPreferences` to define it as the state vector simulator. 

To define the quantum program's execution preferences, use `ExecutionPreferences` and then `set_quantum_program_execution_preferences`. In this example, we will perform a simulation with `num_shots=1` since the state vector simulator performs an exact simulation of the quantum program.

If no backend is defined on the preferences, then the [Classiq simulator](https://docs.classiq.io/latest/user-guide/execution/cloud-providers/classiq-backends/) is selected.


In [ ]:
from classiq.execution import ClassiqBackendPreferences, ExecutionPreferences

backend_preferences = ClassiqBackendPreferences(backend_name = "simulator_statevector")

execution_preferences = ExecutionPreferences(num_shots=1,backend_preferences=backend_preferences)

set_quantum_program_execution_preferences(qprog,execution_preferences)

Now, execute the quantum program using `execute`.

In [18]:
results_statevector = execute(qprog)

The outputs of the quantum program can be obtained through the attributes `state_vector` and `parsed_state_vector` from `results.result_value()`.

In [23]:
print("Output of the executing using counts: ", results_statevector.result_value().state_vector)

print("Output of the executing using parsed_counts: ", results_statevector.result_value().parsed_state_vector)

Output of the executing using counts:  {'000000000': (-7.400485595177172e-17-6.538578924868412e-17j), '000000001': (3.341970536630311e-17+1.8477457316115607e-17j), '000000010': (-1.794722030954402e-17-9.319651695651109e-18j), '000000011': (7.5702391234863e-18-2.669093447656495e-18j), '000000100': (-6.591489994116684e-18+7.286391100256902e-19j), '000000101': (5.820945523978765e-18-4.222844394396122e-18j), '000000110': (-2.2963268189950947e-18+5.1841161125423325e-18j), '000000111': (4.997794902295465e-19-1.1809114559496003e-17j), '000001000': (-2.5162583618884327e-15+0.3535533905932734j), '000001001': (4.871747722757313e-17-2.5919188840442424e-17j), '000001010': (-1.146930866754276e-16-4.0888259362424125e-16j), '000001011': (1.6096377148412664e-18-3.92891977544287e-18j), '000001100': (1.3931499860411522e-16+2.817624409119773e-16j), '000001101': (-6.544202642582989e-19-8.616388045197535e-19j), '000001110': (-9.731879791210046e-17+7.301853502396495e-17j), '000001111': (-3.856102446817961e-

The outputs from the execution obtained via statevector simulator will differ from the default simulator:

* `state_vector` will output a `dict` containing the bitstrings followed by its numerically evaluated amplitudes.
* `parsed_state_vector` will output a `list` of `states`, each containing the values of `x` and `y` followed by its bitstrings and its numerically evaluated amplitudes.